# 🧠 Step 1E — Computation Graph & Autograd (The Heart of Learning)

You already know how to manipulate tensors — but tensors alone can’t “learn.”
What gives PyTorch its magic is autograd and the computation graph.

## 🧩 1️⃣ What Is a Computation Graph?

Whenever you perform mathematical operations on tensors in PyTorch,
it secretly builds a graph of operations.

* The nodes of this graph are tensors (values).

* The edges are operations (like addition, multiplication, etc.).

So if you do:

x = torch.tensor(2.0, requires_grad=True)
y = x * 3
z = y + 5


PyTorch internally builds:

x ──(×3)──► y ──(+5)──► z

### 🧮 2️⃣ What Happens During .backward()

When you call z.backward(), PyTorch walks backward through that graph,
applying the chain rule of calculus to compute the derivative
of z with respect to every tensor that had requires_grad=True.

So here:

* dz/dx = 3

* That value gets stored in x.grad.

## ⚙️ 3️⃣ The “requires_grad” Flag

This flag tells PyTorch:

“Track everything that happens to this tensor so you can compute gradients later.”

If it’s False (the default), PyTorch ignores that tensor during .backward().

In [1]:
import torch
x = torch.tensor(2.0, requires_grad=True)
y = x * 3
z = y + 5

In [2]:
a = torch.tensor(5.0, requires_grad=True)  # tracked
b = torch.tensor(4.0)   

If you mix them, autograd still works, but only variables with requires_grad=True
will receive .grad values.

### 🚦 4️⃣ Why We Need Gradients

Gradients tell us the direction and magnitude to adjust parameters  
to minimize a loss (error).  

Example intuition:  

If loss decreases when weight w decreases, the gradient for w is positive →  

so optimizer subtracts a small step from w.  


### 🧠 5️⃣ Example: One-variable computation

In [26]:
import torch

x = torch.tensor(2.0, requires_grad=True)
y = x ** 3 + 2 * x + 1
y.backward()          # compute dy/dx
print(x.grad)         # → 3x² + 2 = 3*4 + 2 = 14

tensor(14.)


In [27]:
y = x ** 3 + 2 * x + 1
y.backward()          # compute dy/dx
print(x.grad)  

tensor(28.)


### 🔄 6️⃣ What if you call backward multiple times?

Gradients accumulate by default — they don’t reset automatically.  

Otherwise, you’ll see .grad keep increasing over epochs.

In [28]:
x.grad.zero_()   # always reset before next backward

# OR

x = x.detach()

# Both stop gradient tracking temporarily.


## 🧩 7️⃣ Disabling Autograd (for evaluation/inference)

When you’re not training (for example, testing a model),
you don’t need gradient tracking — it saves memory & computation.

In [29]:
# Use either:

with torch.no_grad():
    prediction = model(x)


# or

x = x.detach()


# Both stop gradient tracking temporarily.

NameError: name 'model' is not defined

## 🔍 8️⃣ Behind the Scenes (a tiny mental model)

Each tensor that participates in autograd has:

tensor.data → its raw value

tensor.grad → its computed gradient (after backward)

tensor.grad_fn → a reference to the operation that created it

Check it out:

In [31]:
x = torch.tensor(3.0, requires_grad=True)
y = x * 2
print(y.grad_fn)   # prints something like <MulBackward0 object>

# That’s literally PyTorch keeping a “recipe” to reverse the computation.

## 🧩 9️⃣ Mini Exercises (try these slowly)

### 1️⃣ Create x = torch.tensor(2.0, requires_grad=True)
Compute y = 3*x**2 + 4*x + 5
Find x.grad after .backward() — verify manually.

### 2️⃣ Repeat but call .backward() twice without zeroing grad.
What happens to x.grad?

In [70]:
x = torch.tensor(2.0, requires_grad=True)

In [71]:
y = 3*x**2 + 4*x + 5

In [72]:
y.backward()

In [73]:
x.grad

tensor(16.)

### 3️⃣ Wrap your computation in with torch.no_grad(): and check —
does x.grad update anymore?

🧩 3️⃣ Perfect mental model 💭

You can think of autograd as a notebook keeping track of every math step you do.
When you call backward(), PyTorch flips through the notebook in reverse, applying the chain rule.

But when you use torch.no_grad(), you’re saying:

“Don’t write anything in the notebook for a while.”

So afterward, when you ask it to differentiate, there’s literally no record of the steps to go back through.

#### ⚙️ 4️⃣ Why we still use torch.no_grad()

Even though it blocks gradient tracking, it’s very useful when you’re:

evaluating / testing models

generating outputs (no training)

saving memory (since graph tracking uses extra memory)

preventing accidental gradient updates

In [76]:
before = x.grad.clone()
with torch.no_grad():
    y = 3*x**2 + 4*x + 5
    y.backward()    # has no effect
print(torch.equal(x.grad, before))  # should be True

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

#### causes the error because:

with torch.no_grad(): tells PyTorch:

“Do not build the computation graph for any operation inside this block.”

So when you do  

y = 3*x**2 + 4*x + 5,  

that y does not know it came from x.  

It’s just a plain tensor, not connected to x.grad_fn anymore.

Hence, when you call .backward(), PyTorch says:

“Sorry, this tensor isn’t part of a computation graph — there’s nothing to differentiate!”

That’s literally what

“does not require grad and does not have a grad_fn”
means.

#### so what you are telling me is in no grad function we won't be storing the graph of operations so the backward differentiation can't be done since it doesn't or can't get the prev operation that is done on it

You are telling PyTorch:

“Hey, for the operations inside this block, don’t bother recording how the result was computed.
Just give me the output values, I don’t need gradients later.”

So PyTorch skips building the computation graph —
no grad_fn objects are attached, no connections are made, nothing to trace later.

In [56]:
x.grad

tensor(48.)

### 4️⃣ Create a = torch.tensor([2.0,3.0], requires_grad=True)
Compute b = (a**2).sum()  

Find a.grad. (Hint: gradient will be [2a₁, 2a₂].)

In [77]:
a = torch.tensor([2.0, 3.0], requires_grad=True)

In [78]:
b = (a**2).sum()

In [79]:
b.backward()

In [80]:
a.grad

tensor([4., 6.])

# 🧠 `.backward()` and Autograd — How They Relate

✅ **Yes.**
`autograd` is the **system**, and `.backward()` is the **action** that triggers it.

Let’s think of it like this 👇

| Analogy            | Meaning                                                                                                                 |
| ------------------ | ----------------------------------------------------------------------------------------------------------------------- |
| 🧩 **Autograd**    | The **engine** that records operations and knows how to compute gradients (it builds and stores the computation graph). |
| ⚙️ **.backward()** | The **command** that tells the autograd engine: “OK, now walk this graph in reverse and compute the derivatives.”       |

So:

```python
# Autograd system is *watching* everything
x = torch.tensor(2.0, requires_grad=True)
y = x**2 + 3*x + 1

# This line tells the autograd engine to start differentiation
y.backward()
```

### Step by step:

1️⃣ `requires_grad=True`
→ Autograd starts **recording** all ops done on `x`.

2️⃣ When you compute `y = ...`,
→ Autograd builds a **computation graph** connecting `x → y`.

3️⃣ When you call `y.backward()`,
→ Autograd **traverses the graph backward**, applying the **chain rule** to compute `dy/dx`.

4️⃣ The result is stored in

```python
x.grad
```

✅ So `.backward()` **uses autograd** internally.
Autograd ≈ the engine; `.backward()` ≈ pressing the “differentiate now” button.

---

# 🔍 Example showing autograd & backward in action

```python
import torch

x = torch.tensor(3.0, requires_grad=True)
y = 2*x**3 + 4*x + 1     # autograd tracks this computation
print(y.grad_fn)         # shows grad function, means it's tracked

y.backward()             # backward() calls autograd internally
print(x.grad)            # gradient: dy/dx = 6x^2 + 4 = 6*9 + 4 = 58
```

Here:

* `grad_fn` shows that `y` has a recorded operation (proof autograd is tracking).
* `y.backward()` triggers autograd’s engine to compute the gradient.
* `.grad` holds the computed result.

---

# 💡 Summary (one-line mental map)

> **Autograd = system that builds the graph.**
> **.backward() = trigger that makes autograd compute all gradients.**

---

You’ve now fully connected **tensors → computation graph → autograd → backward() → gradients** ✅
That’s the complete conceptual chain of “how neural networks actually learn.”

---